In [60]:
import pandas as pd

adr_info = pd.read_csv('../data/raw/adr_info.csv')
adr_info['adr_ticker'] = adr_info['adr'].str.replace(' US Equity','')
betas = pd.read_csv('../data/processed/market_etf_hedge_ratios.csv',index_col=0).iloc[-1].rename('beta')

In [61]:
adr_info = adr_info.merge(betas, left_on='adr_ticker', right_index=True, how='left')
adr_info['fx_exposure'] = adr_info['beta'] - 1
adr_info['currency'] = adr_info['currency'].str.replace('GBp','GBP')
adr_info = adr_info.set_index('adr_ticker')
all_fx_exposure = []
for currency in ['GBP','EUR','JPY']:
    all_fx_exposure.append(adr_info['fx_exposure'].where(adr_info['currency']==currency, 0).rename(currency))

exposure_df = pd.concat(all_fx_exposure, axis=1).fillna(0)

adr_ticker
TAK     -0.545026
SONY     0.071162
TM       0.184570
HMC      0.093681
MUFG     0.195123
SMFG     0.231757
MFG      0.282166
BUD     -0.426820
AEG     -0.133202
ARGX    -0.572577
ASML     0.242441
AZN     -0.043830
BCS      0.401657
BTI     -0.501573
BBVA     0.244141
BHP      0.014173
BP      -0.119117
DEO     -0.283998
E       -0.338642
EQNR    -0.597379
ERIC    -0.161793
FMS     -0.419700
GMAB    -0.196772
GSK     -0.071379
HLN     -0.170982
HSBC     0.123418
IHG     -0.194148
ING      0.068414
LYG      0.343471
NGG     -0.203987
NOK     -0.195234
NVS     -0.563854
NVO     -0.081381
NWG      0.406573
PHG      0.068075
PUK      0.273852
PSO     -0.257336
RELX    -0.060929
RIO     -0.059084
RTO      0.213083
RYAAY   -0.042379
SNY     -0.334718
SAN      0.312625
SAP     -0.060820
SHEL    -0.136580
SNN     -0.086985
TS      -0.223020
TTE     -0.356926
UBS     -0.076204
UL      -0.430657
VOD      0.016532
WDS     -0.008998
WPP      0.119707
DB       0.297747
Name: fx_exposure

In [ ]:
from linux_xbbg import blp
#MONTH_CODES = 'FGHJKMNQUVXZ'
MONTH_CODES = 'HMUZ'
code = 'NH'
for code in ['Z ','VG','NH']:
  df = blp.bdh([code + month + str(year) + ' Index'
                for month in MONTH_CODES for year in range(10)],
                  'PX_VOLUME',
                  '20250101',
                  '20250115',
  )

In [114]:
code = 'VG'
df = blp.bdh([code + month + '21' + ' Index'
            for month in MONTH_CODES for year in range(10)],
                'PX_VOLUME',
                '20210301',
                '20210330',
)

EmptyDataError: No columns to parse from file

In [122]:
df = blp.bdh(['VGH25 Index'],
                'PX_VOLUME',
                '20250301',
                '20250310',
)

In [123]:
df

,VGH25 Index
,PX_VOLUME
2025-03-03,983849.0
2025-03-04,1168818.0
2025-03-05,976452.0
2025-03-06,949740.0
2025-03-07,878342.0
2025-03-10,943594.0


In [ ]:
import os

adr_info = pd.read_csv(os.path.join('..', 'data', 'raw', 'adr_info.csv'))
codes = adr_info['index_future_bbg'].unique().tolist()

all_volume_dfs = {}
for code in codes:
    volume_df = blp.bdh([code + month + str(year) + ' Index'
            for month in MONTH_CODES for year in range(10)],
            'PX_VOLUME',
            '20220309',
            '20220330',
    ).droplevel(1,1)
    all_volume_dfs[code] = volume_df

combined_volume_df = pd.concat(all_volume_dfs, axis=1)

In [112]:
combined_volume_df.groupby(level=0, axis=1).apply(lambda x: x.T.droplevel(0,0).idxmax())

/tmp/ipykernel_86988/1936687908.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  combined_volume_df.groupby(level=0, axis=1).apply(lambda x: x.T.droplevel(0,0).idxmax())


,NH,VG,Z
2025-03-10,NHH5 Index,VGH5 Index,Z H5 Index
2025-03-11,NHH5 Index,VGH5 Index,Z H5 Index
2025-03-12,NHH5 Index,VGH5 Index,Z H5 Index
2025-03-13,NHM5 Index,VGH5 Index,Z H5 Index
2025-03-14,NHM5 Index,VGH5 Index,Z H5 Index
2025-03-17,NHM5 Index,VGH5 Index,Z H5 Index
2025-03-18,NHM5 Index,VGH5 Index,Z H5 Index
2025-03-19,NHM5 Index,VGH5 Index,Z H5 Index
2025-03-20,NHM5 Index,VGM5 Index,Z M5 Index
2025-03-21,NHM5 Index,VGM5 Index,Z M5 Index


In [126]:
futures_symbols

,exchange_symbol,bloomberg_symbol,first_rate_symbol,index
0,Z,Z,FTUK,UKX
1,FCE,CF,FCE,CAC
2,FDAX,GX,FDAX,DAX
3,FESX,VG,FESX,SX5E
4,FTI,EO,FTI,AEX
5,FXXP,SXO,FXXP,SXXP
6,NIY,NH,NIY,NKY


In [166]:
import re

futures_symbols = pd.read_csv(os.path.join('..', 'data', 'raw', 'futures_symbols.csv'))
futures_map = futures_symbols.set_index('exchange_symbol')['bloomberg_symbol'].to_dict()
def eurex_to_bbg(symbol):
    MONTH_CODES = 'FGHJKMNQUVXZ'
    month_map = {
        'JAN': 'F',
        'FEB': 'G',
        'MAR': 'H',
        'APR': 'J',
        'MAY': 'K',
        'JUN': 'M',
        'JUL': 'N',
        'AUG': 'Q',
        'SEP': 'U',
        'OCT': 'V',
        'NOV': 'X',
        'DEC': 'Z',
    }
    s = symbol.split()
    if len(s) == 1: # spread
        instrument,_,front_code,back_code,_ = symbol.split('.')
        front_month = month_map[front_code[:3]]
        front_year = front_code[3:4]
        back_month = month_map[back_code[:3]]
        back_year = back_code[3:4]

        if instrument in futures_map:
            bbg_instrument = futures_map[instrument]
        else:
            raise ValueError(f'Unrecognized ICE instrument: {instrument}')

        bbg = (bbg_instrument + front_month + front_year +
               bbg_instrument + back_month + back_year + ' Index')
        
    elif len(s) == 4: # outright
        instrument,_,maturity,_ = symbol.split()
        month = MONTH_CODES[int(maturity[4:5])]
        year = maturity[3:4]

        if instrument in futures_map:
            bbg_instrument = futures_map[instrument]
        else:
            raise ValueError(f'Unrecognized ICE instrument: {instrument}')

        bbg = bbg_instrument + month + year + ' Index'
    else:
        raise ValueError(f'Unrecognized Eurex instrument format: {symbol}')

    return bbg

def parse_ice_maturity(maturity_code):
    month = maturity_code[2]
    year = re.split(r'[-_!]', maturity_code, maxsplit=1)[0][-2:]

    return month, year

def ice_to_bbg(symbol):
    s = symbol.split()
    instrument = s[0]
    if instrument in futures_map:
        bbg_instrument = futures_map[instrument]
    else:
        raise ValueError(f'Unrecognized ICE instrument: {instrument}')
    
    if len(s) == 2: # outright
        maturity_code = s[1]
        month, year = parse_ice_maturity(maturity_code)
        bbg_symbol = bbg_instrument + month + year + ' Index'
    elif len(s) == 3: # spread
        front_code, back_code = s[1:]
        front_month, front_year = parse_ice_maturity(front_code)
        back_month, back_year = parse_ice_maturity(back_code)
        bbg_symbol = (bbg_instrument + front_month + front_year + ' '
                   + bbg_instrument + back_month + back_year + ' Index')
    else:
        raise ValueError(f'Unrecognized ICE instrument format: {instrument}')

    return bbg_symbol

def year_digit_to_two_digit(year_digit):
    year_int = int(year_digit)
    if year_int < 7:
        return '2' + year_digit
    else:
        return '1' + year_digit

def cme_to_bbg(symbol):
    s = symbol.split('-')
    if len(s) == 1: # outright
        year = year_digit_to_two_digit(s[0][-1])
        month = s[0][-2]
        instrument = s[0][:-2]

        if instrument in futures_map:
            bbg_instrument = futures_map[instrument]
        else:
            raise ValueError(f'Unrecognized CME instrument: {instrument}')

        bbg_symbol = bbg_instrument + month + year + ' Index'

    elif len(s) == 2: # spread
        front_symbol, back_symbol = s
        front_year = year_digit_to_two_digit(front_symbol[-1])
        front_month = front_symbol[-2]
        front_instrument = front_symbol[:-2]
        back_year = year_digit_to_two_digit(back_symbol[-1])
        back_month = back_symbol[-2]
        back_instrument = back_symbol[:-2]

        if instrument in futures_map:
            bbg_front_instrument = futures_map[front_instrument]
            bbg_back_instrument = futures_map[back_instrument]
        else:
            raise ValueError(f'Unrecognized CME instrument: {instrument}')
        
        bbg_symbol = (bbg_front_instrument + front_month + front_year + 
                      bbg_back_instrument + back_month + back_year + ' Index')
    else:
        raise ValueError(f'Unrecognized CME instrument format: {symbol}')
                        
    return bbg_symbol

In [152]:
df = pd.read_parquet('../data/raw/futures/bbo-1m/exchange=XEUR.EOBI/')

In [180]:
s = df['symbol'].iloc[-1000:].apply(eurex_to_bbg).unique().tolist()

In [ ]:
blp.bdh(s, ['PX_BID'], '20251201','20251220')

In [ ]:
a = blp.bdh(df['symbol'].apply(ice_to_bbg).unique().tolist(), ['PX_LAST'], '2025-01-01', '2025-12-15')

In [130]:
symbol = 'Z   FMH0025-Z   FMM0025'    
s = symbol.split()
instrument = s[0]
if instrument in futures_map:
    bbg_instrument = futures_map[instrument]
else:
    raise ValueError(f'Unrecognized ICE instrument: {instrument}')

if len(s) == 2: # outright
    maturity_code = s[1]
    month, year = parse_ice_maturity(maturity_code)
    bbg = bbg_instrument + month + year
elif len(s) == 3: # spread
    front_code, back_code = s[1:]
    front_month, front_year = parse_ice_maturity(front_code)
    back_month, back_year = parse_ice_maturity(back_code)
    bbg_symbol = (bbg_instrument + front_month + front_year + ' ' 
            + bbg_instrument + back_month + back_year + ' Index')
else:
    raise ValueError(f'Unrecognized ICE instrument format: {instrument}')

TypeError: can only concatenate str (not "list") to str

In [ ]:
re.split(r'[-_!]', 'FMH0025-Z', maxsplit=1)[0]

'FMH0025'